In [28]:
import numpy as np

In [29]:
#Randomized SVD
def rSVD(X, r, q, p):
    # p - oversampling factor
    # Random Matrix (X_columns x (r + p))
    ny = X.shape[1]
    P = np.random.randn(ny, r + p)

    Z = X @ P

    # Power Iterations    
    for k in range(q):
        Z = X @ (X.T @ Z)
    
    # QR Descomposition
    # Q Orthogonal Matrix
    # R Upper Triangular Matrix
    Q, R = np.linalg.qr(Z, mode='reduced')
    
    Y = Q.T @ X
    
    # Much faster because Y shape is (r + p) x X_columns  
    UY, S, VT = np.linalg.svd(Y, full_matrices=0)
    
    U = Q @ UY

    # S, VT are the same of the Y-SVD
    return U, S, VT

In [30]:
class Avl:

    def __init__(self, value, fila):
      self.value = value
      self.left = None
      self.right = None
      self.filas = [fila]

    def add(self, value, fila):
        if self.value < value:
            if self.right:
                self.right.add(value, fila)
            else:
                self.right = Avl(value, fila)

        elif self.value > value:
            if self.left:
                self.left.add(value, fila)
            else:
                self.left = Avl(value, fila)
            
        else:
            self.filas.append(fila)

    def toArray(self):
        if not self.left and not self.right:
            return [(self.value, self.filas)]

        elif not self.left:
            l = [(self.value, self.filas)]
            l.extend(self.right.toArray())
            return l

        elif not self.right:
            l = self.left.toArray()
            l.append((self.value, self.filas))
            return l

        else:
            l = self.left.toArray()
            l.append((self.value, self.filas))
            l.extend(self.right.toArray())
            return l

In [31]:
# |X1 - X2| and then sum all the components
def simil(x1, x2):

    sum = 0

    for i in range(len(x1)):
        sum += abs(x1[i] - x2[i])

    return sum

In [32]:
#Precalculated matrixes of the Netflix Database

#A has a size of 8GB if your RAM is not enough plis select the other choice
A = np.load('value.enr.npy')
#A = np.load('value.enr.npy', mmap_mode='r')

# Matrixes created using Randomized SVD
U = np.load('U2.enr.npy')
S = np.load('S2.enr.npy')
VT = np.load('VT2.enr.npy')
V = VT.T

# Projection of all vectors from A in V  "A @ V"
L = np.load('L.enr.npy')

# New User Vector with the data of movies he likes/ doesn´t like
X = [0] * 4499

X[7] = -1
X[12] = 1
X[47] = 1
X[190] = 1
X[289] = 1
X[1343] = 1
X[1802] = -1
X[2456] = -1
X[2916] = 1

In [33]:
# Projection of X in V
X_L = X @ V

# Array that will contain the users who like the same kind of movies as the new user
simils = []

# Sorted components of X_L
items = [X_L[0]]
# Reference to the original index of the component
indexes = [0]

# Add the components of X_L in items keeping a descending order
for i in range(1, len(X_L), 1):
    added = False
    for j in range(len(items)):
        if X_L[i] > items[j]:
            items.insert(j, X_L[i])
            indexes.insert(j, i)
            added = True
            break

    if not added:
        items.append(X_L[i])
        indexes.append(i)


# He likes that movie that much as me or a little bit (0.01) less than me?
def it_likes(l, index):
    if l[indexes[index]] >= items[index] or items[index] - l[indexes[index]] <= 0.01:
        return True

    return False

# Add in simils the users that like the two main kind of movie preferred by the new user
for i in range(len(L)):
    if it_likes(L[i], 0) and it_likes(L[i], 1):
        simils.append(i)


# I keep only the users who gave the same ratings for the movies that the new user watch or if they don´t watch it 

indexes = simils
passs = []

for i in range(len(X)):
    if X[i] != 0:
        for j in indexes:
            if A[j][i] == 0 or A[j][i] == X[i]:
                passs.append(j)

        indexes = passs 
        passs = []

# Project X in U
Sd = np.diag(S)
X_proj = X @ V @ np.linalg.inv(Sd)

# Add similarity of each vector of U with X_proj (closer to 0 better), in the AVL
avl = Avl(simil(U[indexes[0]], X_proj), indexes[0])

for fila in range(1, len(indexes), 1):
    avl.add(simil(U[indexes[fila]], X_proj), indexes[fila])

# List of tuples order by similarity, ascending
sort = avl.toArray()
# I keep only the better 50
sort = sort[:50]

# New vector X that will contain the predicted data
new_X = []

for i in range(len(X)):
    if X[i] != 0:
         new_X.append(X[i])
    else:

        # to predict the rating 
        # sum the user rating multiplied by his similarity(inverse because the more important the user is, higher will be his contribution to the sum)

         sum = 0

         for s in sort:
             for f in s[1]:
                 sum += A[f][i] * (1 / s[0])
               
         if abs(sum) <= 0.1:
            new_X.append(0)
         elif sum > 0:
            new_X.append(1)
         elif sum < 0:
            new_X.append(-1)

# Print Results

for i in range(len(new_X)):
    if new_X[i] == 1:
        print(str(i + 1) + ' + 1')
    if new_X[i] == -1:
        print(str(i + 1) + ' - 1')

8 - 1
13 + 1
48 + 1
60 + 1
101 + 1
191 + 1
209 + 1
224 + 1
227 + 1
268 + 1
281 + 1
290 + 1
295 + 1
345 + 1
418 + 1
457 + 1
468 + 1
477 + 1
506 + 1
580 - 1
606 + 1
631 + 1
666 - 1
668 + 1
674 + 1
722 + 1
743 + 1
759 + 1
809 + 1
846 - 1
861 + 1
890 + 1
937 + 1
943 + 1
955 + 1
964 + 1
967 - 1
1013 + 1
1027 + 1
1092 + 1
1206 + 1
1230 + 1
1240 + 1
1251 + 1
1305 - 1
1344 + 1
1357 + 1
1441 + 1
1482 + 1
1484 + 1
1506 + 1
1588 + 1
1741 + 1
1743 + 1
1771 + 1
1803 - 1
1926 + 1
2045 + 1
2112 + 1
2130 + 1
2186 + 1
2189 + 1
2200 + 1
2213 + 1
2217 + 1
2374 - 1
2422 - 1
2432 + 1
2457 - 1
2566 + 1
2612 + 1
2756 + 1
2769 + 1
2809 + 1
2888 + 1
2906 + 1
2916 - 1
2917 + 1
2997 + 1
3025 + 1
3123 - 1
3138 + 1
3182 + 1
3187 + 1
3238 - 1
3298 + 1
3312 + 1
3332 + 1
3348 + 1
3354 + 1
3385 + 1
3439 - 1
3441 + 1
3446 + 1
3486 + 1
3506 - 1
3567 - 1
3689 + 1
3706 + 1
3794 + 1
3841 + 1
3854 + 1
3870 + 1
3946 - 1
3966 + 1
4008 + 1
4010 + 1
4043 + 1
4078 + 1
4177 - 1
4207 + 1
4216 + 1
4293 - 1
4355 + 1
4377 - 1
4384 + 